# 处理Checked Exception的三种方法
---

```java
import java.io.File;
import java.io.FileInputStream;

public class CheckedException {
    public void readFile() {
        String fileName = "file does not exist";
        File file = new File(fileName);
        // 编译器报错, Unhandled exception: java.io.FileNotFoundException
        FileInputStream stream = new FileInputStream(file);
    }
}
```

## 1) Try Catch

Simply wrap the Java code which throws the checked exception within a try catch block. With this approach it's very easy to swallow the exception and then carry on like nothing happened. 

```java
import java.io.File;
import java.io.FileInputStream; import java.io.FileNotFoundException;

class CheckedException {
    public void readFile() {
        String fileName = "file does not exist";
        File file = new File(fileName);
        try {
            // 注意, 在IDEA, 编译器警告⚠️: 'FileInputStream' used without 'try'-with-resources statement 
            FileInputStream stream = new FileInputStream(file);
        } catch (FileNotFoundException e) {
            e.printStackTrace();
        }
    }
}
```

## 2) Throws

We use the keyword throws to throw the checked exception up the stack to the calling method to handle. This is what `FileInputStream()` has just done to you(这句话什么意思?). This looks and feels great - no messy exception code we are writing and we no longer need to handle this exception as someone else can deal with it. The calling method then needs to do something with it ... maybe throw again.

```java
import java.io.File;
import java.io.FileInputStream; 
import java.io.FileNotFoundException;
 
class CheckedException {
    public void readFile() throws FileNotFoundException {
        String fileName = "file does not exist";
        File file = new File(fileName);
        // // 注意, 在IDEA编译器依然给出警告⚠️: 'FileInputStream' used without 'try'-with-resources statement 
        FileInputStream stream = new FileInputStream(file);
    }
}
```

对上面***This is what `FileInputStream` has just done to you***的解释, 看一下源码就知道了:

```java
public FileInputStream(File file) throws FileNotFoundException {
    ...
}
```

意思就是说, 看着throws很不错, 你不用处理, 直接把异常抛给caller, 但是caller 可能也是这么想的, 因此就会陷入循环, 使程序臃肿, 该出手时就得出手. 比如我们刚刚调用了`FileInputStream()`, 而`FileInputStream()`就是把`FileNotFoundException`抛给我们(caller). 这也就是为什么有人会说**遇到不可恢复的异常, 应该立即终止程序运行**. 

## 3) 包装成Runtime Exception, 抛给caller

注意`FileNotFoundException`是checked exception.

```java
import java.io.File;
import java.io.FileInputStream;
import java.io.FileNotFoundException;

class ProcessingException extends RuntimeException {
    public ProcessingException(String message) {
        super(message);
    }
}

class CheckedException {
    public void readFile() {
        String fileName = "file does not exist";
        File file = new File(fileName);
        try {
            FileInputStream stream = new FileInputStream(file);
        } catch (FileNotFoundException e) {
            throw new ProcessingException("Error opening file");
        }
    }
}
```

---

上面说到了遇到了不可恢复的异常, 就应该立即终止程序运行, 那么终止运行是什么意思呢?

你看, checked exception必须要用`try catch`block或者`throw`给caller, 如果用`try catch`包裹, 那JVM就不会终止程序运行(因为你处理了异常), 如果继续`throw`给调用者显然不符合我们的原则, 因为这个异常本就是不可修复的, 所以即使你传给别人又能干嘛呢? 

你看说到底, 还是针对checked exception说的, 因为编译器不要求runtime exception必须被捕获, 不被捕获的异常才会导致程序terminate, 所以我们就把checked exception包装成runtime exception, 然后调用者也不用捕获, 这时候发生异常, 就直接程序终止了.

问题来了, 怎么包装? 就是上面处理checked exception三种方法的第三种方法, 自定义一个runtime exception就行了. 

看看[别人怎么说的](https://rollbar.com/blog/how-to-handle-checked-unchecked-exceptions-in-java/):

Below is an example that is very commonly used in micro service architecture. If we received a request and we cannot, say, read data from our database needed for this request, the database will throw us a **checked exception**, maybe an `SQLException` or something similar. Because this data is important, we cannot fulfil this request without it.

This means **there is nothing we can actually do with this exception that can fix the problem**, but if we do nothing the code will carry on its execution regardless.

We could throw the exception to the calling code until we get to the top of the chain and return the exception to the user. By doing that we are then littering all the layers above with an exception that they really do not care about, nor should they. **What we really want is an unchecked exception** to terminate this request gracefully. 这种情况就是我们有checked exception, 我们想要unchecked exception.

```java
import java.io.File;
import java.io.FileInputStream;
import java.io.FileNotFoundException;

class ProcessingException extends RuntimeException {
    public ProcessingException(String message) {
        super(message);
    }
}

class CheckedException {
    public void readFile() {
        String fileName = "file does not exist";
        File file = new File(fileName);
        try {
            FileInputStream stream = new FileInputStream(file);
        } catch (FileNotFoundException e) {
            throw new ProcessingException("Error opening file");
        }
    }
}
```

# 说一些其他的
---

其实我们可以根据java提供的异常处理方式来反推checked exception和unchecked exception(runtime). 

对于checked exception, 最终必须要用`try catch`来捕捉(因为即使你刚开始不捕捉, 而是throw给其他caller, 那最终总会有一个caller来catch这个checked exception, 否则编译就无法通过, 不考虑把checked exception包装成runtime exception的情况). 那这说明了什么呢, 这说明了checked exception是可以被修复的(大部分), 比如出现`ClassNotFoundException`, `FileNotFoundException`或者`SocketException` etc, 我们可以通过提示用户来重新输入文件名或者尝试重新连接网络来处理异常从而正确连接, 这也是recoverable conditions中的一种.

对于unchecked exception, Java并不强制要求我们捕捉, 这意味着我们可以用`if`逻辑判断语句来规避, 比如NPE或者是一些算数异常(除数为0), 我们可以用`if(a == 0)`类似这种来规避. 

> A runtime exception is a programming error and is fatal whereas a checked exception is an exception condition within your code’s logic and can be recovered or re-tried from. https://rollbar.com/blog/how-to-handle-checked-unchecked-exceptions-in-java/

# Java设计出checked exception有必要吗？
---

“的确，checked 异常在业界是有争论的，它假设我们捕获了异常，并且针对这种情况作了相应的处理，但有些时候，根本就没法处理。”我说，“就拿上面提到的 `ClassNotFoundException` 异常来说，我们假设对其进行了 `try-catch`，可真的出现了 `ClassNotFoundException` 异常后，我们也没多少的可操作性，再 `Class.forName()` 一次？”

另外，checked 异常也不兼容函数式编程，后面如果你写 Lambda/Stream 代码的时候，就会体验到这种苦涩。


当然了，checked 异常并不是一无是处，尤其是在遇到 IO 或者网络异常的时候，比如说进行 Socket 链接，我大致写了一段：

```java
import java.io.IOException;
import java.net.Socket;

public class Demo2 {
    private String mHost;
    private int mPort;
    private Socket mSocket;
    private final Object mLock = new Object();

    public void run() {
    }

    private void initSocket() {
        while (true) {
            try {
                Socket socket = new Socket(mHost, mPort);
                synchronized (mLock) {
                    mSocket = socket;
                }
                break;
            } catch (IOException e) {
                e.printStackTrace();
            }
        }
    }
}
// 注意 try catch在while loop里面, 出现了异常也只是打印一下信息, 程序并不会终止, 因为你捕捉到了. 
```

当发生 IOException 的时候，socket 就重新尝试连接，否则就 break 跳出循环。意味着如果 IOException 不是 checked 异常，这种写法就略显突兀(因为我们有个原则比如对于NPE这种应该使用if逻辑判断去规避异常)，但IOException 没办法像 ArithmeticException 那样用一个 if 语句判断除数是否为 0 去规避, 所以编译器强制我们catch unchecked exception还是有点道理的. 

或者说，强制性的 checked 异常可以让我们在编程的时候去思考，遇到这种异常的时候该怎么更优雅的去处理。显然，Socket 编程中，肯定是会遇到 IOException 的，假如 IOException 是非检查型异常，就意味着开发者也可以不考虑，直接跳过，交给 Java 虚拟机来处理，但我觉得这样做肯定更不合适。